In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import fbprophet
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv("/kaggle/input/monthly-milk-production-pounds/monthlyMilkProduction.csv")
df.head()

# Task 1: Data Preprocessing With Time Series

In [ ]:
df.tail()

In [ ]:
#drop the bad record
df.drop(168, axis=0, inplace=True)

In [ ]:
df.tail()

In [ ]:
#rename the column
df.columns = ["ds", "y"]     #month --> ds(dates) & milk production --> y(value)


In [ ]:
df.head()

In [ ]:
df.plot()

In [ ]:
#convert date to yyyy-dd-mm
df['ds'] = pd.to_datetime(df['ds'])

In [ ]:
df.head() 


In [ ]:
df['y'].plot()

In [ ]:
df['y'].plot()

Stationary data means constant "Variance"

In [ ]:
#import facebook Prophet

from fbprophet import Prophet

In [ ]:
dir(Prophet)

# Task 2: Intialize the model

In [ ]:
model = Prophet()
model.fit(df)

In [ ]:
model  # by default model will give monthly_seasonality


In [ ]:
model.seasonalities

In [ ]:
# create some future dataframes(dates) of 365 days
#this data is used to make predictions
future_dates = model.make_future_dataframe(periods=365)

In [ ]:
future_dates

In [ ]:
# Predictions
predictions = model.predict(future_dates)

In [ ]:
predictions.head()

In [ ]:
predictions[['ds', 'yhat','yhat_lower', 'yhat_upper']].tail()

# Task 3 : Plot the predicted projection

Blue line ---> the predicted value.

Black dot ---> real data in our dataset

Light Blue --> trends

In [ ]:
model.plot(predictions)

# Task 4 : Visualize each componennt

In [ ]:
model.plot_components(predictions)

# Task 5: Cross Validation

we will do this by comparing the predicted values with actual values. 

Select Cut off Points in the history of data then fit model with data upto that cut off point.

Then we compare the actual values to the predicted values. 

This Takes following parameter:
    1. horizon --> the forcast horizon
    2. initial size of initial training period
    3. period the spacing between cutoff dates

In [ ]:
from fbprophet.diagnostics import cross_validation

In [ ]:
df_cv = cross_validation(model, horizon = "365 days", period='180 days', initial = '1095 days')

In [ ]:
df_cv.head()

In [ ]:
df_cv.tail()

In [ ]:
#performance matrix
from fbprophet.diagnostics import performance_metrics
df_performance = performance_metrics(df_cv)
df_performance.head()

In [ ]:
df_performance

In [ ]:
from fbprophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(df_cv, metric='rmse')

Thhank you